In [3]:
#!pip install langchain-community

# Question Answering for Documents using RAG

In [4]:
!pip install langchain
!pip install pypdf
!pip install chromadb
!pip install openai
!pip install tiktoken
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 4.2 MB/s eta 0:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 845.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
#!pip install langchain-community

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import openai
from openai import OpenAI
import os

In [ ]:
pdf_file_path = "/content/drive/MyDrive/Deep_Learnings.pdf"
embedding_persistent_directory = "/content/drive/MyDrive/sample_data/embedding_vector"

In [ ]:
client = OpenAI(api_key="11WTgs1uqEETp3x6ry0kbm8ez5q7SrMeQj9pZY6s4GbYUlvCgVU6JQQJ99BCACHYHv6XJ3w3AAAAACOGFRO6")

In [ ]:
loader = PyPDFLoader(pdf_file_path)
pages = loader.load()
len(pages)

44

Chunking PDF documents

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(pages)
len(chunks)

71

In [ ]:
chunks[5].page_content

'and ambiguous. Baidu – one of the leading search engines of China – has developed a voice \nrecognition system that is faster and more accurate than humans at producing text on a mobile phone. \nIn both English and Mandarin.\nWhat is particularly fascinating, is that generalizing the two languages didn’t require much additional \ndesign effort: “Historically, people viewed Chinese and English as two vastly different languages, and \nso there was a need to design very different features,” Andrew Ng says, chief scientist at Baidu. “The \nlearning algorithms are now so general that you can just learn.”\nGoogle is now using deep learning to manage the energy at the company’s data centres. They’ve cut \ntheir energy needs for cooling by 40%. That translates to about a 15% improvement in power usage \nefficiency for the company and hundreds of millions of dollars in savings\nNeural Network Overview And Its Use Case'

## Create embedding and store it in cromadb

In [ ]:
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-10-4250a910d430>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warn

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
db = Chroma.from_documents(documents=chunks, embedding=embedding, persist_directory=embedding_persistent_directory)

In [ ]:
query = "what is convolution neural network"

# Lets use similarity serach from langcahin and understand it how it works. Lets check with the above query.

In [ ]:
chunks_with_query_context = db.similarity_search_with_score(query, k=2)

In [ ]:
print("Serach Result")
for i, r in enumerate(chunks_with_query_context,1):
  print(r[0].page_content)
  print("\n\n\n")


Serach Result
of a convolution filter, and then each FeatureMap There are multiple neurons.




of a convolution filter, and then each FeatureMap There are multiple neurons.






Now this is the time for writing systematics code for this, with the help of two helper function.

1. for semantic serach in chroma_db.
2. for genrative model, to take the input (query, and semantic search result), and return the genrated output with openai/ or ny other llm.

In [ ]:
def gen_answer(question, context):
  contex = context
  debug = False
  if debug:
    print("Context \n"+context)
    print("\n\n")
  try:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Anser the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\n",
            },
            {
                "role": "user",
                "content": f"Context: {contex}\n\n---\n\nQuestion: {question}\nAnswer:",
            },
        ],
        temperature=0,
        max_tokens=500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
    )
    return response.choices[0].message.content
  except Exception as e:
    print(f"An error occured : {str(e)}")


In [ ]:
def search_chromadb(db, question, k):
  try:
    if not db:
      raise ValueError("Database object is not provide")
    if not question or not isinstance(question, str):
      raise ValueError("Invalid question input")
    if not isinstance(k, int) or k <= 0:
      raise ValueError("Invalid value for k")

    docs = db.similarity_search(question, k=k)
    if not docs:
      print("No relevant documents found")
      return []
    result = [doc.page_content for doc in docs if doc.page_content]
    if not result:
      print("No non-empty results from semantic search")

    print(f"chunks from semantic search : {result}")
    return result
  except Exception as e:
    print(f"An error occured : {str(e)}")
    return []

In [ ]:
loaded_db = Chroma(persist_directory=embedding_persistent_directory, embedding_function=embedding)

<ipython-input-17-f916b989bcac>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  loaded_db = Chroma(persist_directory=embedding_persistent_directory, embedding_function=embedding)


In [ ]:
print(f"\n##############################\n")
print(" My first Question Answering System ")
print(f"\n##############################\n")
while True:
  user_query = input("Enter your question(type 'quit' or 'Q' or 'q' to exit): ").strip().lower()
  if user_query == "quit" or user_query == "q" or user_query == "Q":
    print(f"Exiting the question answer system")
    break
    print("Exiting the program.")
    break
  context = search_chromadb(loaded_db, user_query, 1)
  answer = gen_answer(user_query, context)
  print(f"GENAIAnswer : {answer}")
  print(f"\n##############################\n")


##############################

 My first Question Answering System 

##############################

Enter your question(type 'quit' or 'Q' or 'q' to exit): what is activation function
chunks from semantic search : ["based on some threshold. Activation function also filters out the important data.\nHere each neuron passes some information to others neuron then others neuron take this information\nand do some calculation and pass to other , that's how it makes the neuron networks."]
An error occured : Error code: 401 - {'error': {'message': 'Incorrect API key provided: 11WTgs1u************************************************************************FRO6. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
GENAIAnswer : None

##############################

Enter your question(type 'quit' or 'Q' or 'q' to exit): q
Exiting the question answer system
